# Design

This is an decoder model which will take in a roundness value and output a pseudoword that corresponds to the roundness value

In [1]:
from utils.pseudoword_generator import *
import pandas as pd
import torch

pd.set_option('display.max_columns', None)
device = "cuda" if torch.cuda.is_available() else "cpu"
state = 42

VERSION = 3

c:\Users\qkm20\anaconda3\envs\pseudoword\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
# Import dataset

data = pd.read_csv('datasets/japanese_pseudowords.csv')
data

,Pseudoword,Roundness
0,kineratote,0.150207
1,rutsusajibo,0.110635
2,neza,0.468633
3,poutano,0.326936
4,uyoshipasu,0.131503
...,...,...
9995,gechigapaba,0.077712
9996,ragi,0.351221
9997,mugugo,0.234996
9998,rureto,0.280292


In [3]:
data.describe()

,Roundness
count,10000.000000
mean,0.259523
std,0.196518
min,0.038889
25%,0.090111
50%,0.248405
75%,0.345424
max,0.860149


In [4]:
# Split data into train, val and test sets

trn = data.sample(frac=0.8, random_state=state)
val = data.drop(trn.index).sample(frac=0.5, random_state=state)
tst = data.drop(trn.index).drop(val.index)
trn.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)
tst.reset_index(inplace=True, drop=True)

In [5]:
print(f"Train set: {len(trn)} samples, Validation set: {len(val)} samples, Test set: {len(tst)} samples")

Train set: 8000 samples, Validation set: 1000 samples, Test set: 1000 samples


# Model

In [6]:
model = RoundnessToTextModel(
    t5_model_name="sonoisa/t5-base-japanese",
    freeze_t5=False,
    hidden_dim=256,
    output_dim=768,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

In [7]:
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.6)

# Training

In [8]:
train(
    model=model,
    optimizer=optimizer,
    trn_roundness=trn["Roundness"],
    val_roundness=val["Roundness"],
    tst_roundness=tst["Roundness"],
    trn_texts=trn["Pseudoword"],
    val_texts=val["Pseudoword"],
    tst_texts=tst["Pseudoword"],
    batch_size=min(len(val), 100),
    epochs=100,
    patience=10,
    scheduler=scheduler,
)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch   1/100, Train Loss: 8.3036, Validation Loss: 3.4454
Epoch   2/100, Train Loss: 3.5977, Validation Loss: 3.5512
Epoch   3/100, Train Loss: 3.4606, Validation Loss: 3.9083
Epoch   4/100, Train Loss: 3.4155, Validation Loss: 3.7888
Epoch   5/100, Train Loss: 3.3886, Validation Loss: 3.5334
Epoch   6/100, Train Loss: 3.3750, Validation Loss: 3.7611
Epoch   7/100, Train Loss: 3.3519, Validation Loss: 3.2878
Epoch   8/100, Train Loss: 3.3491, Validation Loss: 3.4796
Epoch   9/100, Train Loss: 3.3303, Validation Loss: 3.6421
Epoch  10/100, Train Loss: 3.3156, Validation Loss: 3.4321
Epoch  11/100, Train Loss: 3.3038, Validation Loss: 3.4214
Epoch  12/100, Train Loss: 3.2994, Validation Loss: 3.3588
Epoch  13/100, Train Loss: 3.2885, Validation Loss: 3.3995
Epoch  14/100, Train Loss: 3.2872, Validation Loss: 3.3068
Epoch  15/100, Train Loss: 3.2839, Validation Loss: 3.2702
Epoch  16/100, Train Loss: 3.2706, Validation Loss: 3.2060
Epoch  17/100, Train Loss: 3.2688, Validation Loss: 3.26

# Testing

In [9]:
random_sample = tst.sample(n=10, random_state=42)
for _, row in random_sample.iterrows():
    print(f"Roundness Value : {row['Roundness']}")
    print(f"Original Word   : {row['Pseudoword']}")
    print(f"Predicted word  : {inference(model, row["Roundness"])}")
    print()

Roundness Value : 0.047891214
Original Word   : guidopen
Predicted word  : daniperu

Roundness Value : 0.3033671
Original Word   : kamuyo
Predicted word  : topopo

Roundness Value : 0.060305737
Original Word   : yudozoke
Predicted word  : shifuchiru

Roundness Value : 0.31320328
Original Word   : kafu
Predicted word  : kake

Roundness Value : 0.25200036
Original Word   : gonu
Predicted word  : yazo

Roundness Value : 0.0557759
Original Word   : zezuzuyo
Predicted word  : nurapegu

Roundness Value : 0.063052915
Original Word   : ronibema
Predicted word  : nenokuha

Roundness Value : 0.14619786
Original Word   : dageshike
Predicted word  : mopipikoheyo

Roundness Value : 0.15073475
Original Word   : naguhazanu
Predicted word  : mubizazotape

Roundness Value : 0.060333844
Original Word   : wakirasu
Predicted word  : zukujipe



In [10]:
roundness_list = []
for i in range(11):
    roundness_list.append(i/10)

for roundness in roundness_list:
    print(f"Roundness Value: {roundness}")
    print(f"Predicted word: {inference(model, roundness)}")
    print()

Roundness Value: 0.0
Predicted word: yachihino

Roundness Value: 0.1
Predicted word: kibikamiwa

Roundness Value: 0.2
Predicted word: hapozato

Roundness Value: 0.3
Predicted word: sumu

Roundness Value: 0.4
Predicted word: genumi

Roundness Value: 0.5
Predicted word: meishi

Roundness Value: 0.6
Predicted word: oge

Roundness Value: 0.7
Predicted word: baa

Roundness Value: 0.8
Predicted word: gere

Roundness Value: 0.9
Predicted word: atei

Roundness Value: 1.0
Predicted word: pie



# Save and load model

In [11]:
save_model(model, filename=f"pseudoword_generator_v0{VERSION}.pth")

Model saved to outputs/pseudoword_generator_v03.pth


In [ ]:
model = load_model(filename=f"pseudoword_generator_v0{VERSION}.pth")